In [28]:
import numpy as np
import pandas as pd
from pathlib import Path

In [29]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [30]:
train_df1= train_df.drop(['loan_status', 'index'], axis=1)
train_df1.drop(['Unnamed: 0'],axis='columns', inplace=True)
traindum = pd.get_dummies(train_df1)
X_train = traindum

In [31]:
# Convert categorical data to numeric and separate target feature for testing data
testdf = test_df.copy()
testdf.head()


,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [32]:
# add missing dummy variables to testing set
testdf = testdf.drop(['loan_status', 'index'], axis=1)
testdf.drop(['Unnamed: 0'], axis=1, inplace=True)
testdum = pd.get_dummies(testdf)
testdum['debt_settlement_flag_Y']=0


testdum.head()
X_test = testdum

My prediction is the the logistical regression model will be more accurate. I believe that the number of variables and the numeric data in this data set will make the random forrest model less accurate.Random forrest model performs better with catagorial non numeric data. Being this is a financial data set variables are numerical I predict the logistical regression model on the scaled data will be the most accurate. 

In [33]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression

y_train = train_df['loan_status']
y_test = test_df['loan_status']


classifier = LogisticRegression(solver='lbfgs',max_iter=200000)
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)
print(f'Logistic regression model accuracy: {100 * accuracy_score(y_test, y_pred):.3f}%')

Training Data Score: 0.6954844006568145
Testing Data Score: 0.5720969800085071
Logistic regression model accuracy: 57.210%


In [38]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train, y_train)
print(f'Testing Score: {clf.score(X_test, y_test)}')

Testing Score: 0.6433432581880051


In [35]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Logistical Regression with scaling is the best model it had higher level of accuracy for both the training and test data. This model had an accuracy of .76 for the test data which is significantly higher then the test data score of the Random Forrest Classifier which was .64

In [36]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier.fit(X_train_scaled, y_train)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test)}")


Training Data Score: 0.7080459770114943
Testing Data Score: 0.7679710761378137


In [37]:
# Train a Random Forest Classifier model on the scaled data and print the model score
print(f'Testing Score: {clf.score(X_train_scaled, y_train)}')
print(f'Training Score: {clf.score(X_test_scaled, y_test)}')

Testing Score: 0.5
Training Score: 0.5


Random Forrest on scaled (.5) and unscaled data (.64) was less accurate than the logistic regression model. Unscaled data for logistical regression Training (.69) and Testing (.57) which means the Logisitic Regression on Scaled data Training (.70) and Testing (.76) was the most accurate. While the scaled logistic regression model was the most accurate with a .76 accuracy in this type of business I would want a model with a higher level of accuracy due to the fact the less accurate the model the more money I stand to lose when the model does not accurately reflect the credit risk of a bad/high risk borrower. As I predicted the Logistic Regression with scaleing is the most accurate model. 